In [3]:
!pip install PyPortfolioOpt
# Import packages
!pip install yfinance
!pip install yahoo_fin
!pip install scikit-learn
!pip install matplotlib
!pip install textblob
!pip install seaborn
!pip install pypfopt
!pip install statsmodels
!pip install tqdm

import yfinance as yf
import pandas as pd
from yahoo_fin import stock_info
from yahoo_fin.stock_info import get_analysts_info
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import seaborn as sns
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pypfopt (from versions: none)
ERROR: No matching distribution found for pypfopt


In [4]:
# Implement a Monte Carlo simulation for the portfolio of recommended stocks
def monte_carlo_simulation(recommended_stocks, num_simulations=1000, time_horizon=252):
    simulated_portfolios = []
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')['Adj Close']
        returns = historical_prices.pct_change().dropna()
        avg_return = returns.mean()
        std_dev = returns.std()

        for _ in range(num_simulations):
            simulation = [historical_prices.iloc[-1]]
            for _ in range(time_horizon):
                next_price = simulation[-1] * (1 + np.random.normal(avg_return, std_dev))
                simulation.append(next_price)
            simulated_portfolios.append(simulation)

    return simulated_portfolios

In [5]:
# Perform Monte Carlo simulation and plot results
simulated_portfolios = monte_carlo_simulation(recommended_stocks)
plt.figure(figsize=(14, 7))
for portfolio in simulated_portfolios:
    plt.plot(portfolio, color='blue', alpha=0.1)
plt.title('Monte Carlo Simulation of Portfolio')
plt.xlabel('Time (days)')
plt.ylabel('Portfolio Value')
plt.show()

NameError: name 'recommended_stocks' is not defined

In [ ]:
# Calculate Value at Risk (VaR) for the portfolio
def calculate_var(recommended_stocks, confidence_level=0.95):
    portfolio_values = []
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')['Adj Close']
        returns = historical_prices.pct_change().dropna()
        portfolio_values.extend(returns)

    portfolio_values = np.sort(portfolio_values)
    index = int((1 - confidence_level) * len(portfolio_values))
    var = portfolio_values[index]
    return var

In [ ]:

# Calculate and print Value at Risk
var = calculate_var(recommended_stocks)
print(f'\nValue at Risk (VaR) at 95% confidence level: {var}')

In [ ]:
# Advanced feature: Portfolio optimization using the Efficient Frontier
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

def portfolio_optimization(recommended_stocks):
    prices = yf.download([symbol for symbol, _ in recommended_stocks], period='5y')['Adj Close']
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)

    return cleaned_weights

In [ ]:
# Perform portfolio optimization and print results
optimized_weights = portfolio_optimization(recommended_stocks)
print('\nOptimized Portfolio Weights:')
for symbol, weight in optimized_weights.items():
    print(f"{symbol}: {weight:.4f}")

In [ ]:
# Begining of additional features
# 1. Economic Indicators Analysis
def fetch_economic_indicators():
    indicators = {
        'GDP': 21.43e12,
        'Unemployment Rate': 3.5,
        'Inflation Rate': 2.3,
        'Interest Rate': 0.25
    }
    return indicators

In [ ]:
# Fetch and print economic indicators
economic_indicators = fetch_economic_indicators()
print('\nEconomic Indicators:')
for indicator, value in economic_indicators.items():
    print(f"{indicator}: {value}")

In [ ]:
# 2. Industry Sector Analysis
def analyze_sector_performance(tickers):
    sector_performance = {}
    for sector in tickers['GICS Sector'].unique():
        sector_tickers = tickers[tickers['GICS Sector'] == sector]['Symbol']
        sector_data = prepare_data(pd.DataFrame({'Symbol': sector_tickers}))
        sector_performance[sector] = sector_data['Adj Close'].mean()
    return sector_performance

In [ ]:
# Analyze and print sector performance
sector_performance = analyze_sector_performance(tickers)
print('\nSector Performance:')
for sector, performance in sector_performance.items():
    print(f"{sector}: {performance}")


In [ ]:
# 3. Advanced Technical Indicators
def calculate_technical_indicators(recommended_stocks):
    technical_indicators = {}
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')
        historical_prices['SMA'] = historical_prices['Adj Close'].rolling(window=50).mean()
        historical_prices['EMA'] = historical_prices['Adj Close'].ewm(span=50, adjust=False).mean()
        technical_indicators[symbol] = historical_prices[['SMA', 'EMA']]
    return technical_indicators


In [ ]:
# Calculate and print technical indicators
technical_indicators = calculate_technical_indicators(recommended_stocks)
for symbol, indicators in technical_indicators.items():
    print(f'\nTechnical Indicators for {symbol}:')
    print(indicators.tail())

In [ ]:
# 4. Sector Rotation Strategy
def sector_rotation_strategy(sector_performance, money):
    sorted_sectors = sorted(sector_performance.items(), key=lambda x: x[1], reverse=True)
    top_sectors = sorted_sectors[:3]
    allocation = money / len(top_sectors)
    return {sector: allocation for sector, _ in top_sectors}

In [ ]:
# Implement sector rotation strategy and print results
sector_rotation = sector_rotation_strategy(sector_performance, money)
print('\nSector Rotation Strategy Allocation:')
for sector, allocation in sector_rotation.items():
    print(f"{sector}: ${allocation}")

In [ ]:
# 5. Mean Reversion Strategy
def mean_reversion_strategy(recommended_stocks):
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')
        prices = historical_prices['Adj Close']
        mean_price = prices.mean()
        std_dev = prices.std()
        upper_band = mean_price + 2 * std_dev
        lower_band = mean_price - 2 * std_dev
        plt.figure(figsize=(14, 7))
        plt.plot(prices, label=symbol)
        plt.axhline(mean_price, color='red', linestyle='--', label='Mean')
        plt.axhline(upper_band, color='green', linestyle='--', label='Upper Band')
        plt.axhline(lower_band, color='green', linestyle='--', label='Lower Band')
        plt.legend()
        plt.title(f'Mean Reversion Strategy for {symbol}')
        plt.show()


In [ ]:
# Plot mean reversion strategy for recommended stocks
mean_reversion_strategy(recommended_stocks)

In [ ]:
# 6. Sentiment-Based Trading Strategy
def sentiment_based_trading_strategy(recommended_stocks):
    sentiments = {}
    for stock, _ in recommended_stocks:
        news_articles = fetch_news_articles(stock)
        sentiment_scores = sentiment_analysis(news_articles)
        avg_sentiment = np.mean(list(sentiment_scores.values()))
        sentiments[stock] = avg_sentiment
    return sentiments

In [ ]:

# Implement sentiment-based trading strategy and print results
sentiment_scores = sentiment_based_trading_strategy(recommended_stocks)
print('\nSentiment-Based Trading Strategy:')
for stock, sentiment in sentiment_scores.items():
    print(f"{stock}: {sentiment}")

In [ ]:
# 7. Dividend Yield Analysis
def dividend_yield_analysis(recommended_stocks):
    dividend_yields = {}
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')
        dividends = yf.Ticker(symbol).dividends
        avg_dividend_yield = dividends.sum() / historical_prices['Adj Close'].mean()
        dividend_yields[symbol] = avg_dividend_yield
    return dividend_yields

In [ ]:
# Calculate and print dividend yields
dividend_yields = dividend_yield_analysis(recommended_stocks)
print('\nDividend Yields:')
for symbol, yield_ in dividend_yields.items():
    print(f"{symbol}: {yield_}")
# Calculate and print dividend yields
dividend_yields = dividend_yield_analysis(recommended_stocks)
print('\nDividend Yields:')
for symbol, yield_ in dividend_yields.items():
    print(f"{symbol}: {yield_}")


In [ ]:
# 8. Earnings and Revenue Analysis
def earnings_and_revenue_analysis(recommended_stocks):
    earnings_data = {}
    for symbol, _ in recommended_stocks:
        analysts_info = get_analysts_info(symbol)
        earnings_data[symbol] = analysts_info['Earnings Estimate']
    return earnings_data

In [ ]:
# Fetch and print earnings and revenue data
earnings_data = earnings_and_revenue_analysis(recommended_stocks)
print('\nEarnings and Revenue Analysis:')
for symbol, data in earnings_data.items():
    print(f"{symbol}: {data}")

In [ ]:
# 9. Market Sentiment Index (MSI)
def calculate_market_sentiment_index(recommended_stocks):
    sentiment_index = 0
    for stock, _ in recommended_stocks:
        news_articles = fetch_news_articles(stock)
        sentiment_scores = sentiment_analysis(news_articles)
        avg_sentiment = np.mean(list(sentiment_scores.values()))
        sentiment_index += avg_sentiment
    sentiment_index /= len(recommended_stocks)
    return sentiment_index


In [ ]:
# Calculate and print Market Sentiment Index
market_sentiment_index = calculate_market_sentiment_index(recommended_stocks)
print(f'\nMarket Sentiment Index: {market_sentiment_index}')


In [ ]:
# 10. Relative Strength Index (RSI)
def calculate_rsi(recommended_stocks):
    rsi_values = {}
    for symbol, _ in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')
        delta = historical_prices['Adj Close'].diff()
        gain = (delta.where(delta > 0, 0)).mean()
        loss = (-delta.where(delta < 0, 0)).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        rsi_values[symbol] = rsi
    return rsi_values

In [ ]:
# Calculate and print RSI values
rsi_values = calculate_rsi(recommended_stocks)
print('\nRelative Strength Index (RSI):')
for symbol, rsi in rsi_values.items():
    print(f"{symbol}: {rsi}")

In [ ]:
# 11. Predictive Analytics for Economic Indicators
def predictive_analytics_economic_indicators(economic_indicators):
    predictions = {}
    for indicator, value in economic_indicators.items():
        model = RandomForestRegressor()
        X = np.arange(10).reshape(-1, 1)  # Placeholder for time series data
        y = np.random.rand(10)  # Placeholder for indicator values
        model.fit(X, y)
        future_values = model.predict(np.array([[11], [12], [13]]))
        predictions[indicator] = future_values
    return predictions

In [ ]:
# Predict future economic indicators and print results
predicted_economic_indicators = predictive_analytics_economic_indicators(economic_indicators)
print('\nPredictive Analytics for Economic Indicators:')
for indicator, predictions in predicted_economic_indicators.items():
    print(f"{indicator}: {predictions}")

In [ ]:
# 12. Advanced Machine Learning Models
from sklearn.ensemble import GradientBoostingRegressor

def advanced_machine_learning_models(X_train, y_train, X_test, y_test):
    gbm = GradientBoostingRegressor()
    gbm.fit(X_train, y_train)
    y_pred = gbm.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return gbm, mse

In [ ]:
# Train and evaluate advanced machine learning model
advanced_model, advanced_mse = advanced_machine_learning_models(X_train, y_train, X_test, y_test)
print(f'\nAdvanced Model Mean Squared Error: {advanced_mse}')

In [ ]:
# 13. Volatility Index (VIX) Analysis
def analyze_vix():
    vix = yf.download('^VIX', period='5y')['Adj Close']
    vix_trend = vix.rolling(window=30).mean()
    return vix_trend


In [ ]:
# Analyze and plot VIX trend
vix_trend = analyze_vix()
plt.figure(figsize=(14, 7))
plt.plot(vix_trend, label='VIX Trend')
plt.title('Volatility Index (VIX) Analysis')
plt.xlabel('Date')
plt.ylabel('VIX Value')
plt.legend()
plt.show()

In [ ]:
# 14. Cryptocurrency Analysis
def cryptocurrency_analysis():
    cryptos = ['BTC-USD', 'ETH-USD', 'LTC-USD']
    crypto_data = {}
    for crypto in cryptos:
        historical_prices = yf.download(crypto, period='5y')
        crypto_data[crypto] = historical_prices
    return crypto_data

In [ ]:
# Fetch and print cryptocurrency data
crypto_data = cryptocurrency_analysis()
print('\nCryptocurrency Data:')
for crypto, data in crypto_data.items():
    print(f"{crypto}:")
    print(data.tail())


In [ ]:
# 15. Advanced Risk Management Techniques
def advanced_risk_management(portfolio_weights, recommended_stocks):
    prices = yf.download([symbol for symbol, _ in recommended_stocks], period='5y')['Adj Close']
    returns = prices.pct_change().dropna()
    cov_matrix = returns.cov()
    portfolio_variance = np.dot(portfolio_weights.T, np.dot(cov_matrix, portfolio_weights))
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

In [ ]:
# Calculate and print portfolio volatility
portfolio_volatility = advanced_risk_management(np.array(list(optimized_weights.values())), recommended_stocks)
print(f'\nPortfolio Volatility: {portfolio_volatility}')

In [6]:
# Fetch the list of S&P 500 companies
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [7]:
# Function to get live data for companies
def get_live_data_for_companies(tickers):
    live_data_for_companies = {}
    for symbol in tqdm(tickers['Symbol']):
        try:
            live_data = yf.Ticker(symbol).info
            live_data_for_companies[symbol] = live_data
        except:
            print(f"Could not fetch data for {symbol}")
    return live_data_for_companies

In [8]:
# Get live data for S&P 500 companies
live_data_sp500 = get_live_data_for_companies(tickers)

# User input for investment details
money = float(input('Enter the amount of money: '))
time = int(input('Enter the time in weeks: '))
risk_percentage = float(input('Enter risk percentage (0-100): '))

# Validate risk percentage input
if not (0 <= risk_percentage <= 100):
    raise ValueError('Invalid input for risk percentage. Please enter a value between 0 and 100.')

returns = float(input('Enter returns (1-100): '))

100%|██████████| 503/503 [01:00<00:00,  8.27it/s]


Enter the amount of money: 100000
Enter the time in weeks: 10
Enter risk percentage (0-100): 50
Enter returns (1-100): 50


In [9]:
# Create a DataFrame with the user input
user_data = pd.DataFrame({
    'money': [money],
    'time': [time],
    'risk_percentage': [risk_percentage],
    'returns': [returns]
})

print('\nUser Input:')
print(user_data)


User Input:
      money  time  risk_percentage  returns
0  100000.0    10             50.0     50.0


In [ ]:
# Prepare historical data for model training
def prepare_data(tickers):
    historical_data = []
    for symbol in tqdm(tickers['Symbol']):
        try:
            stock_data = yf.download(symbol, period='2y')
            stock_data['Symbol'] = symbol
            historical_data.append(stock_data)
        except:
            print(f"Could not download data for {symbol}")
    return pd.concat(historical_data)

historical_data = prepare_data(tickers)

In [12]:

# Feature engineering
historical_data['year'] = historical_data.index.year
historical_data['month'] = historical_data.index.month
historical_data['day'] = historical_data.index.day
historical_data = historical_data.dropna()



In [13]:
# Label encode the stock symbols
le = LabelEncoder()
historical_data['symbol_encoded'] = le.fit_transform(historical_data['Symbol'])

# Define features and target variable
features = historical_data[['symbol_encoded', 'year', 'month', 'day', 'Open', 'High', 'Low', 'Close', 'Volume']]
target = historical_data['Adj Close']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor()


In [14]:
# Define hyperparameters for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
    }

In [ ]:
# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(rf_regressor, param_grid, cv=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)


In [18]:
# Get the best model from grid search
best_rf_model = grid_search.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
# Make predictions on the test set
y_pred = best_rf_model.predict(X_test)

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')


In [ ]:
# Function to predict stock investment based on user input
def predict_stock_investment(model, user_input, le):
    user_input['symbol_encoded'] = le.transform(['AAPL'])[0]  # Placeholder, change to desired logic
    user_input = user_input[['money', 'time', 'risk_percentage', 'symbol_encoded']]
    predicted_returns = model.predict(user_input)
    return predicted_returns

In [ ]:
# User input for prediction
user_input = pd.DataFrame({
    'money': [money],
    'time': [time],
    'risk_percentage': [risk_percentage]
})


In [ ]:
# Predict returns using the machine learning model
predicted_returns = predict_stock_investment(best_rf_model, user_input, le)
print(f'Predicted Returns: {predicted_returns}')

In [ ]:
# Function to recommend stocks based on predicted returns and risk percentage
def recommend_stocks(live_data_sp500, predicted_returns, risk_percentage, money):
    recommended_stocks = []
    for symbol, data in live_data_sp500.items():
        try:
            if risk_percentage < 33 and data['beta'] < 1:
                recommended_stocks.append((symbol, data))
            elif 33 <= risk_percentage < 66 and 1 <= data['beta'] < 1.5:
                recommended_stocks.append((symbol, data))
            elif risk_percentage >= 66 and data['beta'] >= 1.5:
                recommended_stocks.append((symbol, data))
        except:
            pass
    return recommended_stocks


In [ ]:
# Recommend stocks based on predicted returns and risk percentage
recommended_stocks = recommend_stocks(live_data_sp500, predicted_returns, risk_percentage, money)


In [ ]:
# Print recommended stocks and reasons for selection
print('\nRecommended Stocks:')
for symbol, data in recommended_stocks:
    print(f"\nSymbol: {symbol}")
    print(f"Company: {data['longName']}")
    print(f"Beta: {data['beta']}")
    print(f"Market Cap: {data['marketCap']}")
    print(f"Industry: {data['industry']}")
    print(f"Reason: {'Low risk' if risk_percentage < 33 else 'Medium risk' if risk_percentage < 66 else 'High risk'} stock with appropriate beta value.")


In [ ]:
# Function to fetch news articles for recommended stocks
def fetch_news_articles(symbol):
    try:
        news = stock_info.get_news(symbol)
        return news[:5]  # Return the top 5 news articles
    except:
        return []

In [ ]:
# Fetch news articles for recommended stocks
for stock, _ in recommended_stocks:
    news_articles = fetch_news_articles(stock)
    print(f'\nNews for {stock}:')
    for article in news_articles:
        print(f"Title: {article['title']}\nLink: {article['link']}\n")



In [ ]:
# Function to plot predicted returns for recommended stocks
def plot_predicted_returns(recommended_stocks, model, user_input, le):
    plt.figure(figsize=(14, 7))
    for symbol, data in recommended_stocks:
        user_input['symbol_encoded'] = le.transform([symbol])[0]
        user_input = user_input[['money', 'time', 'risk_percentage', 'symbol_encoded']]
        predicted_returns = model.predict(user_input)
        plt.plot(user_input['time'], predicted_returns, label=symbol)

    plt.title('Predicted Returns for Recommended Stocks')
    plt.xlabel('Time (weeks)')
    plt.ylabel('Predicted Returns')
    plt.legend()
    plt.show()

# Plot predicted returns for recommended stocks
plot_predicted_returns(recommended_stocks, best_rf_model, user_input, le)

In [ ]:
# Calculate the Sharpe ratio for each recommended stock
def calculate_sharpe_ratio(recommended_stocks):
    sharpe_ratios = {}
    risk_free_rate = 0.01  # 1% risk-free rate for calculation
    for symbol, data in recommended_stocks:
        try:
            historical_prices = yf.download(symbol, period='5y')['Adj Close']
            returns = historical_prices.pct_change().dropna()
            avg_return = returns.mean()
            std_dev = returns.std()
            sharpe_ratio = (avg_return - risk_free_rate) / std_dev
            sharpe_ratios[symbol] = sharpe_ratio
        except:
            print(f"Could not calculate Sharpe ratio for {symbol}")
    return sharpe_ratios

In [ ]:
# Calculate and print Sharpe ratios
sharpe_ratios = calculate_sharpe_ratio(recommended_stocks)
print('\nSharpe Ratios:')
for symbol, ratio in sharpe_ratios.items():
    print(f"{symbol}: {ratio}")

In [ ]:

# Sentiment analysis of news articles for recommended stocks
def sentiment_analysis(news_articles):
    sentiments = {}
    for article in news_articles:
        analysis = TextBlob(article['title'])
        sentiments[article['title']] = analysis.sentiment.polarity
    return sentiments

In [ ]:
# Perform sentiment analysis and print results
for stock, _ in recommended_stocks:
    news_articles = fetch_news_articles(stock)
    sentiments = sentiment_analysis(news_articles)
    print(f'\nSentiment Analysis for {stock}:')
    for title, sentiment in sentiments.items():
        print(f"Title: {title}\nSentiment Polarity: {sentiment}\n")

In [ ]:
# Plot cumulative returns for recommended stocks
def plot_cumulative_returns(recommended_stocks):
    plt.figure(figsize=(14, 7))
    for symbol, data in recommended_stocks:
        historical_prices = yf.download(symbol, period='5y')['Adj Close']
        cumulative_returns = (historical_prices / historical_prices.iloc[0]) - 1
        plt.plot(cumulative_returns, label=symbol)

    plt.title('Cumulative Returns for Recommended Stocks')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.show()

In [ ]:
# Plot cumulative returns for recommended stocks
plot_cumulative_returns(recommended_stocks)